# 13 - LCEL 管道语法

## 用途
学习 LangChain 1.0 通过 "|" 操作符创建 LCEL 流水线，这是最重要和推荐的写法

## 学习目标
- 掌握管道操作符 "|" 语法
- 理解数据流向和传递机制
- 能构建完整的通义千问处理链
- 掌握 prompt|llm|parser 核心模式

## 🔑 前置要求
**注意**：需要先完成 Runnable 基础概念学习，理解链式组合原理

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. 管道操作符基础语法

In [ ]:
# 管道操作符基础语法 - 独立代码块
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🔧 管道操作符基础语法理解:")
print("=" * 50)

# 检查 OpenAI API 配置
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ 未找到 OPENAI_API_KEY 环境变量")
    print("   请在 .env 文件中添加: OPENAI_API_KEY=your_key_here")
else:
    print(f"✅ OpenAI API Key 已配置 (长度: {len(openai_api_key)})")
    
    print(f"\n📝 LCEL 管道语法核心概念:")
    print(f"   1. | 操作符：连接不同的 Runnable 组件")
    print(f"   2. 数据流：左到右传递输出结果")
    print(f"   3. 类型安全：自动类型转换和验证")
    print(f"   4. 可组合性：任意组件都可以管道连接")
    
    # 创建基础处理函数
    def step1_process(text: str) -> str:
        return f"步骤1处理: {text}"
    
    def step2_process(text: str) -> str:
        return f"步骤2处理: {text}"
    
    def step3_process(text: str) -> str:
        return f"步骤3处理: {text}"
    
    # 包装为 Runnable
    step1 = RunnableLambda(step1_process)
    step2 = RunnableLambda(step2_process)
    step3 = RunnableLambda(step3_process)
    
    print(f"\n🏗️  创建的基础组件:")
    print(f"   1. 步骤1: {type(step1).__name__}")
    print(f"   2. 步骤2: {type(step2).__name__}")
    print(f"   3. 步骤3: {type(step3).__name__}")
    
    # 基础管道连接
    print(f"\n🔗 基础管道连接测试:")
    
    # 两步管道
    pipe_2step = step1 | step2
    print(f"   两步管道类型: {type(pipe_2step)}")
    
    result_2step = pipe_2step.invoke("测试数据")
    print(f"   两步管道结果: {result_2step}")
    
    # 三步管道
    pipe_3step = step1 | step2 | step3
    print(f"\n   三步管道类型: {type(pipe_3step)}")
    
    result_3step = pipe_3step.invoke("测试数据")
    print(f"   三步管道结果: {result_3step}")
    
    # 验证点：管道操作符基础语法正确
    assert result_2step == "步骤2处理: 步骤1处理: 测试数据", "两步管道结果错误"
    assert result_3step == "步骤3处理: 步骤2处理: 步骤1处理: 测试数据", "三步管道结果错误"
    print(f"\n✅ 验证通过：管道操作符基础语法理解正确")

## 2. prompt | llm 基础管道

In [ ]:
# prompt | llm 基础管道 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🤖 prompt | llm 基础管道测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建 LLM
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=60
        )
        
        # 创建 PromptTemplate
        prompt = PromptTemplate(
            input_variables=["topic"],
            template="请简单介绍一下 {topic}，用一句话概括。"
        )
        
        print(f"📝 创建的组件:")
        print(f"   1. Prompt: {type(prompt).__name__}")
        print(f"   2. LLM: {type(llm).__name__}")
        print(f"   3. 模板变量: {prompt.input_variables}")
        
        # 构建 prompt | llm 管道
        print(f"\n🔗 构建 prompt | llm 管道:")
        
        prompt_llm_chain = prompt | llm
        
        print(f"   管道类型: {type(prompt_llm_chain)}")
        print(f"   管道可用: {prompt_llm_chain is not None}")
        
        # 测试基础调用
        print(f"\n🧪 测试基础调用:")
        
        test_topics = [
            "人工智能",
            "机器学习",
            "深度学习"
        ]
        
        for i, topic in enumerate(test_topics, 1):
            print(f"\n   测试 {i}: 主题 = {topic}")
            
            # 调用管道
            result = prompt_llm_chain.invoke({"topic": topic})
            
            print(f"   输入: {{'topic': '{topic}'}}")
            print(f"   输出类型: {type(result)}")
            print(f"   输出内容: {result.content}")
            
            # 验证输出格式
            assert hasattr(result, 'content'), "输出缺少 content 属性"
            assert len(result.content) > 0, "输出内容为空"
        
        # 验证点：prompt | llm 管道正确构建
        final_test = prompt_llm_chain.invoke({"topic": "Python"})
        assert hasattr(final_test, 'content'), "最终测试输出格式错误"
        
        print(f"\n✅ 验证通过：prompt | llm 基础管道构建正确")
        
    except Exception as e:
        print(f"\n❌ prompt | llm 管道构建失败: {e}")

## 3. prompt | llm | parser 完整链

In [ ]:
# prompt | llm | parser 完整链 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from dotenv import load_dotenv
import os
import json

# 加载环境变量
load_dotenv()

print("🔗 prompt | llm | parser 完整链测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建 LLM
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=80
        )
        
        # 创建解析器
        str_parser = StrOutputParser()
        json_parser = JsonOutputParser()
        
        print(f"📝 创建的组件:")
        print(f"   1. LLM: {type(llm).__name__}")
        print(f"   2. 字符串解析器: {type(str_parser).__name__}")
        print(f"   3. JSON解析器: {type(json_parser).__name__}")
        
        # 1. 字符串输出链
        print(f"\n🔗 1. 字符串输出链 (prompt | llm | StrOutputParser):")
        
        str_prompt = PromptTemplate(
            input_variables=["text"],
            template="请总结以下内容的核心观点：{text}"
        )
        
        str_chain = str_prompt | llm | str_parser
        
        print(f"   链类型: {type(str_chain)}")
        
        test_text = "人工智能技术正在快速发展，深度学习算法不断突破，应用场景越来越广泛。"
        str_result = str_chain.invoke({"text": test_text})
        
        print(f"   输入: {test_text}")
        print(f"   输出类型: {type(str_result)}")
        print(f"   输出内容: {str_result}")
        
        # 2. JSON 输出链
        print(f"\n🔗 2. JSON 输出链 (prompt | llm | JsonOutputParser):")
        
        json_prompt = PromptTemplate(
            input_variables=["text"],
            template="分析以下文本并返回JSON格式：{{\"summary\": \"摘要\", \"keywords\": [\"关键词1\", \"关键词2\"]}}\n文本：{text}"
        )
        
        json_chain = json_prompt | llm | json_parser
        
        print(f"   链类型: {type(json_chain)}")
        
        json_result = json_chain.invoke({"text": test_text})
        
        print(f"   输入: {test_text}")
        print(f"   输出类型: {type(json_result)}")
        print(f"   输出内容: {json.dumps(json_result, ensure_ascii=False, indent=6)}")
        
        # 3. 复杂链组合
        print(f"\n🔗 3. 复杂链组合测试:")
        
        # 创建多步骤处理链
        analysis_prompt = PromptTemplate(
            input_variables=["topic"],
            template="请分析 {topic} 的三个主要特点，用逗号分隔。"
        )
        
        def split_features(text: str) -> list:
            """分割特点列表"""
            return [feature.strip() for feature in text.split(',') if feature.strip()]
        
        from langchain_core.runnables import RunnableLambda
        
        feature_splitter = RunnableLambda(split_features)
        complex_chain = analysis_prompt | llm | str_parser | feature_splitter
        
        complex_result = complex_chain.invoke({"topic": "Python编程语言"})
        
        print(f"   复杂链结果: {complex_result}")
        print(f"   结果类型: {type(complex_result)}")
        print(f"   特征数量: {len(complex_result)}")
        
        # 验证点：完整链正确构建
        assert isinstance(str_result, str), "字符串链输出类型错误"
        assert isinstance(json_result, dict), "JSON链输出类型错误"
        assert isinstance(complex_result, list), "复杂链输出类型错误"
        assert "summary" in json_result, "JSON输出缺少summary字段"
        
        print(f"\n✅ 验证通过：prompt | llm | parser 完整链构建正确")
        
    except Exception as e:
        print(f"\n❌ 完整链构建失败: {e}")

## 4. 数据流向和类型传递

In [ ]:
# 数据流向和类型传递 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📊 数据流向和类型传递测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建 LLM
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=50
        )
        
        # 创建类型检查函数
        def check_type_and_pass(data):
            print(f"   数据类型: {type(data)}")
            print(f"   数据内容: {data}")
            return data
        
        type_checker = RunnableLambda(check_type_and_pass)
        str_parser = StrOutputParser()
        
        print(f"📝 测试组件:")
        print(f"   1. 类型检查器: {type(type_checker).__name__}")
        print(f"   2. 字符串解析器: {type(str_parser).__name__}")
        
        # 1. 基础数据流测试
        print(f"\n🔍 1. 基础数据流测试:")
        
        prompt = PromptTemplate(
            input_variables=["input"],
            template="处理输入: {input}"
        )
        
        # 构建带类型检查的链
        chain_with_checks = prompt | type_checker | llm | type_checker | str_parser | type_checker
        
        print(f"   执行链: prompt → type_checker → llm → type_checker → str_parser → type_checker")
        
        result = chain_with_checks.invoke({"input": "测试数据流"})
        
        print(f"\n   最终结果: {result}")
        print(f"   最终类型: {type(result)}")
        
        # 2. 类型转换测试
        print(f"\n🔍 2. 类型转换测试:")
        
        def dict_to_string(data: dict) -> str:
            """字典转字符串"""
            return f"字符串化: {data.get('text', '未知')}"
        
        def string_to_dict(data: str) -> dict:
            """字符串转字典"""
            return {"result": data, "length": len(data)}
        
        dict_to_str_runnable = RunnableLambda(dict_to_string)
        str_to_dict_runnable = RunnableLambda(string_to_dict)
        
        # 类型转换链
        type_conversion_chain = (
            PromptTemplate(input_variables=["text"], template="{text}")
            | dict_to_str_runnable
            | llm
            | str_parser
            | str_to_dict_runnable
        )
        
        conversion_result = type_conversion_chain.invoke({"text": "类型转换测试"})
        
        print(f"   转换链结果: {conversion_result}")
        print(f"   结果类型: {type(conversion_result)}")
        
        # 3. 数据保持测试
        print(f"\n🔍 3. 数据保持测试 (RunnablePassthrough):")
        
        # 使用 RunnablePassthrough 保持原始数据
        passthrough_chain = (
            {
                "original": RunnablePassthrough(),
                "processed": PromptTemplate(input_variables=["text"], template="处理: {text}") | llm | str_parser
            }
        )
        
        passthrough_result = passthrough_chain.invoke({"text": "数据保持测试"})
        
        print(f"   原始数据: {passthrough_result['original']}")
        print(f"   处理数据: {passthrough_result['processed']}")
        print(f"   结果类型: {type(passthrough_result)}")
        
        # 4. 错误处理测试
        print(f"\n🔍 4. 错误处理测试:")
        
        def error_handler(data):
            """错误处理函数"""
            try:
                if "error" in str(data).lower():
                    raise ValueError("检测到错误关键词")
                return data
            except Exception as e:
                return f"错误处理: {str(e)}"
        
        error_handler_runnable = RunnableLambda(error_handler)
        
        # 带错误处理的链
        safe_chain = (
            PromptTemplate(input_variables=["text"], template="{text}")
            | error_handler_runnable
            | type_checker
        )
        
        normal_result = safe_chain.invoke({"text": "正常数据"})
        error_result = safe_chain.invoke({"text": "包含error的数据"})
        
        print(f"   正常数据处理: {normal_result}")
        print(f"   错误数据处理: {error_result}")
        
        # 验证点：数据流向和类型传递正确
        assert isinstance(result, str), "基础数据流最终类型错误"
        assert isinstance(conversion_result, dict), "类型转换结果类型错误"
        assert isinstance(passthrough_result, dict), "数据保持结果类型错误"
        assert "original" in passthrough_result, "缺少原始数据字段"
        
        print(f"\n✅ 验证通过：数据流向和类型传递正确")
        
    except Exception as e:
        print(f"\n❌ 数据流向测试失败: {e}")

## 5. 学习总结与最佳实践

In [ ]:
# 学习总结与最佳实践 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📋 LCEL 管道语法学习总结:")
print("=" * 50)

# 验证点检查
verification_points = [
    "✅ 管道操作符：| 语法基础",
    "✅ prompt|llm：基础管道组合",
    "✅ prompt|llm|parser：完整处理链",
    "✅ 数据流向：类型传递和转换",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 LCEL 管道语法最佳实践:")
print("1. 管道组合：使用 | 操作符连接组件")
print("2. 数据流向：理解左到右的传递机制")
print("3. 类型安全：注意输入输出类型匹配")
print("4. 解析器使用：StrOutputParser 和 JsonOutputParser")
print("5. 错误处理：在链中加入异常处理逻辑")

print("\n🔧 常用管道模式:")
print("1. prompt | llm - 基础生成")
print("2. prompt | llm | StrOutputParser - 字符串输出")
print("3. prompt | llm | JsonOutputParser - 结构化输出")
print("4. prompt | llm | parser | RunnableLambda - 后处理")
print("5. RunnablePassthrough | prompt | llm - 数据保持")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 RunnablePassthrough 数据透传")
print("2. 掌握流式输出 Streaming 技术")
print("3. 学习 ChatMessageHistory 消息历史")
print("4. 探索更复杂的 LCEL 组合模式")
print("5. 实践生产级管道应用开发")

# 最终验证：确保 LCEL 管道语法基础功能可用
try:
    openai_api_key = os.getenv('OPENAI_API_KEY')
    
    if openai_api_key:
        # 简单测试
        llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=30)
        prompt = PromptTemplate(
            input_variables=["text"],
            template="总结: {text}"
        )
        
        # 构建 prompt | llm | parser 链
        chain = prompt | llm | StrOutputParser()
        result = chain.invoke({"text": "LCEL 管道语法测试"})
        
        print(f"\n🎉 最终验证成功:")
        print(f"   API 状态: 可用")
        print(f"   链类型: {type(chain).__name__}")
        print(f"   输出类型: {type(result)}")
        print(f"   输出内容: {result}")
        print("\n✅ LCEL 管道语法学习完成！")
        
        print(f"\n🎊 核心管道技术掌握！")
        print(f"   已掌握: Runnable + RunnableMap + LCEL 管道语法")
        print(f"   下一步: 学习流式输出和消息历史技术")
        
    else:
        print(f"\n⚠️  最终验证跳过: OpenAI API Key 未配置")
        print("   请配置 OPENAI_API_KEY 后重试")
        
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")